### CS4423 - Networks
Angela Carnevale  
School of Mathematical and Statistical Sciences  
University of Galway

#### 1. Graphs and Graph Theory


# Week 2, Lecture 2: 
# Graphs, Relations and Matrices. Bipartite Graphs.

 Together with `networkx`, we import `numpy` for matrix operations.

In [ ]:
import networkx as nx
import numpy as np
opts={'with_labels':True,'node_color':'y'}

## Graphs are Relations

We have seen that graphs can be interpreted as relations with certain properties:

A **simple** graph with node set $X$ is a **symmetric**, **irreflexive** relation on $X$. 

The relation on the nodes that of *being adjacent*.  

The **adjacency matrix** of a graph is a special case of adjacency matrix of a relation.


## Composition and Adjacency Matrices.

* Relations can be composed, like functions.  

* If $R$ is a relation
from a set $X$ to a set $Y$, and if $S$ is a relation from $Y$ to a set $Z$,
then the __composite relation__ $R \circ S$ is the relation
from $X$ to $Z$, defined by $x (R \circ S) z$ if there is a
an element $y \in Y$ such that $x R y$ and $y S z$.

* The adjacency matrix of the composite relation $R \circ S$
is *essentially* the (matrix) product of the adjacency matrices
of the individual relations $R$ and $S$. 

You can check how this works in practice with a graph on yesterday's notebook. Today, let's see how this works out with the first few responses from our survey.

We'll construct two relations starting from (part of) the data gathered through our survey on TV shows watched. 

By the way, here's a summary of the results from the first 10 respondents:

![TV survey summary](images/TV_shows.png)

... and here's a list of the responses in the form of an adjacency list that can be read by `networkx` to construct a graph:

In [ ]:
!cat data/tv_short.adj

Consider (in order):

* the set of the first ten respondents $X_{10}$: Angela, A1, Evan, A2, A3, Andrew, Erika, M, John, Aoife; and

* The set $Y$ of shows under consideration. In order: Breaking Bad, Better Call Saul, Derry Girls, Succession, How I met your mother, Mr Bean, The Office, Friends, The X files, Lost, The Boys, Game of Thrones.

On  $X_{10}\times Y$ we can consider the relation $R$ given by $xRy$ if respondent $x$ watched show $y$. The corresponding adjacency matrix (obtained from the data above) is

$$ A=\begin{bmatrix}
0&0&1&0&0&0&0&1&0&1&0&0\\
1&0&1&0&1&0&0&0&0&0&0&1\\
0&0&0&0&0&0&1&1&0&0&0&1\\
1&0&1&0&0&0&1&0&0&0&1&1\\
0&0&0&0&0&1&1&0&0&0&0&0\\
1&1&0&0&0&1&1&0&0&0&0&0\\
1&1&0&0&1&1&1&1&0&0&0&1\\
1&0&0&0&1&1&0&1&0&0&0&1\\
0&0&1&0&1&0&1&1&0&0&0&0\\
0&0&1&0&0&1&0&1&0&0&0&0
\end{bmatrix}$$

(here rows are indexed by elements of $X_{10}$ and columns by elements of $Y$).

Here's our adjacency matrix as a `numpy` array

In [ ]:
A=np.array([[0,0,1,0,0,0,0,1,0,1,0,0],
            [1,0,1,0,1,0,0,0,0,0,0,1],
            [0,0,0,0,0,0,1,1,0,0,0,1],
            [1,0,1,0,0,0,1,0,0,0,1,1],
            [0,0,0,0,0,1,1,0,0,0,0,0],
            [1,1,0,0,0,1,1,0,0,0,0,0],
            [1,1,0,0,1,1,1,1,0,0,0,1],
            [1,0,0,0,1,1,0,1,0,0,0,1],
            [0,0,1,0,1,0,1,1,0,0,0,0],
            [0,0,1,0,0,1,0,1,0,0,0,0]])

What if we compose the relation $R:X_{10}\to Y$ with its *transpose* $R^T:Y\to X_{10}$? 

The transpose of the relation $R$ is simply the one that has $A^T$ as adjacency matrix. So at the level of matrices, this means multiplying $A$ by $A^T$, which we can do in `numpy` as follows:

In [ ]:
B=A@A.transpose()
print(B)

How can we interpret the entries of the above matrix in terms of TV shows and respondents?

We've seen that for two relations $R$ and $S$ that can be composed,

* If $A = (a_{ij})$ is the adjacency matrix of $R$, and $B = (b_{jk})$ the adjacency matrix of $S$,
then the $i,k$-entry of the product $AB$ is
$$(AB)_{ik} = \sum_{j} a_{ij} b_{jk},$$
which is exactly the __number__ of elements $y \in Y$ such that $x_i R
y$ and $y S z_k$.  




* This tells us that all it needs for $x_i$ to be $(R \circ S)$-related
to $z_k$ is this number to be at least $1$.  

* Hence, replacing all
nonzero entries in the product matrix $AB$ with $1$ yields the
adjacency matrix of the composite $R \circ S$.

* In our case, the entry in position $(i,j)$ is **exactly the number of shows in common between the $i$th and $j$th respondent**.

* In particular, the entry in position $(i,i)$ tells us how many shows were watched by participant number $i$.

* We can use the observation above to transform the matrix $B$ into an adjacency matrix of a graph.

* In `numpy`, one can use **boolean indexing** and other convenient methods to do so.

First, we simply replace all the entries of $B$ which are stricly greater than $1$ with $1$s:

In [ ]:
B[B>1]=1
print(B)

Then, we fill the diagonal with zeros:

In [ ]:
np.fill_diagonal(B,0)
print(B)

The above matrix is the adjacency matrix of the composite relation. It tells us something about the **common interests** of the $10$ actors under consideration: there is a $1$ in position $(i,j)$ if and only if the $i$th respondent and the $j$th respondent share at least one show on their watchlist. Here, for instance, not every pair of respondents had shows in common.

This is now the adjacency matrix of a homogeneous relation (on $X_{10}$) and we can therefore construct a graph. We'll do so directly from our `numpy` adjacency matrix.

In [ ]:
G=nx.from_numpy_matrix(B)

In [ ]:
nx.draw(G,**opts)

A graph like this one, representing the patterns of intersections of a family of sets, is also known as an [**intersection graph**](https://en.wikipedia.org/wiki/Intersection_graph). We'll encounter this graph in a different guise soon.

Note that we could have used the *transpose* construction (that is, we could have taken $R^T\circ R$):

In [ ]:
C=A.transpose()@A
print(C)

* This time, the $(i,j)$ entry of the matrix tells us the number of people that have watched both the $i$th and the $j$th show. In particular, the diagonal entries give us back the height of each column in the histogram we've seen before:

In [ ]:
np.diagonal(C)

As an exercise, you could construct an adjacency matrix and corresponding graph from the matrix $C$ (see also below).

## Bipartite Graphs

Affiliation networks are convenient ways to represent and store information like that we gathered through our survey. Mathematically, these are are bipartite graphs. Let's see/recall some of their important features.

A (simple) graph $G = (X, E)$ is called **bipartite**, if the vertex set $X$ is a disjoint union
of two sets $X_1$ and $X_2$ so that each edge in $E$ links a vertex in $X_1$ with a vertex in $X_2$. 

We can think of the vertices in the two sets as **coloured** with different colours. For instance, we can think of nodes in $X_1$ as white nodes and those in $X_2$ as black nodes.

Here is a sample bipartite graph $B$, specified to the `Graph` constructor by its edge list.

In [ ]:
edges = [(0,5), (1,5), (1,6), (1,7), (1,8), 
  (2,8), (3,5), (3,9), (4,7), (4,8), (4,9)]
B = nx.Graph(edges)
nx.draw(B, **opts)

In this graph, the **white** nodes can be taken  as the set $X_1 = \{0,1,2,\dots,4\}$ 
and the **black** nodes as $X_2 = \{5,6,\dots,9\}$.
The drawing command `nx.draw` takes as optional argument a dictionary `pos` that specifies for
each node a (relative) position in the drawing.  Here, the node is the key and the 
position is a pair of $x$,$y$-coordinates.  In this example we can use the (integer) quotient
and remainder, as returned by the python method `divmod` to quickly compute a dictionary of positions
that have the white nodes on the left, and the black nodes on the right.

In [ ]:
divmod(7, 5)

In [ ]:
pos = {x: divmod(x, 5) for x in range(10)}
print(pos)

In [ ]:
nx.draw(B, pos, **opts)

Node colors can be specified as a *list* assigned to the keyword argument `node_color`.  We can use the $x$-coordinates of the node positions for that purpose.

In [ ]:
color = [pos[x][0] for x in B.nodes()]
color

In [ ]:
print(B.nodes)

In [ ]:
opts2 = { "with_labels": True, "node_color": color, "font_color": 'r' }
nx.draw(B, **opts2)

In [ ]:
nx.draw(B, pos, **opts2)

A **(vertex)-coloring** of a graph $G$ is an assignment of (finitely many) colors to the nodes of $G$,
so that any two nodes which are connected by an edge have *different* colors.

A graph is called **$N$-colorable**, if it has a vertex coloring with (at most) $N$ colors.

**Theorem.** Let $G$ be a graph.  The following are equivalent:

* $G$ is bipartite;

* $G$ is $2$-colorable;
 
* each cycle in $G$ has even length.

(We'll give precise definitions of **cycle** and **length** in a bit)



2D grids are naturally bipartite:

In [ ]:
G44 = nx.grid_2d_graph(4, 4)
nx.draw(G44)

How would you find a $2$-coloring of this graph?

The method `nx.bipartite.color` determines a $2$-coloring of a graph $G$ algorithmically, if it exists, i.e. if
$G$ is bipartite.

In [ ]:
color = nx.bipartite.color(G44)
color

This won't work on a graph that is not $2$-colorable:

In [ ]:
nx.bipartite.color(nx.complete_graph(3))

In [ ]:
color = [color[x] for x in G44.nodes()]
color

In [ ]:
opts2 = { "with_labels": True, "node_color":color, "font_color":'r' }

In [ ]:
nx.draw(G44, **opts2)

Note how the nodes are labelled in a $2D$ grid in `networkx`...

In [ ]:
G44.nodes()

##  Code Corner

### `python`

* `divmod`: [[doc]](https://docs.python.org/3/library/functions.html#divmod) the built-in quotient-and-remainder

In [ ]:
divmod(-7, 5)

### `Numpy`

* `toarray`:  convert a sparse matrix into a proper array

* `fill_diagonal`: fill the diagonal entries of an array with a given value.

### `networkx`

* `adjacency_matrix` computes the adjacency matrix of a graph

  * `from_numpy_matrix` constructs a graph from its adjacency matrix

##  Exercises

1. Use the `complete_graph` function in `networkx` to construct a $5 \times 5$ matrix
   with entries $0$ on the diagonal and all other entries $1$.

2. Construct an adjacency matrix and a graph from the matrix $C$ above. What does it mean for two vertices in this graph to be adjacent at the level of the TV shows?